In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time

SEARCH_URL = "https://timkiem.vnexpress.net/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Các query muốn tìm
QUERIES = ["bóng đá Việt Nam", "V-League", "cầu thủ Việt Nam", "CLB Việt Nam"]

# Mục tiêu số lượng bài
MAX_ARTICLES = 3000
SAVE_PER_FILE = 500

# Bộ nhớ tạm
RESULTS = []
CRAWLED_URLS = set()
CRAWLED_COUNT = 0
FILE_INDEX = 1

def extract_author(soup):
    # 1. Tác giả trong p.author
    author_tag = soup.select_one("p.author")
    if author_tag:
        return author_tag.get_text(strip=True)

    # 2. Tác giả trong p.author_mail
    author_tag = soup.select_one("p.author_mail")
    if author_tag:
        return author_tag.get_text(strip=True)

    # 3. Tác giả ở đoạn cuối bài
    content_div = soup.select_one("article.fck_detail")
    if content_div:
        paragraphs = content_div.find_all("p")
        if paragraphs:
            last_p = paragraphs[-1].get_text(strip=True)
            if 2 <= len(last_p) <= 50: 
                return last_p

    return "Unknown"

def crawl_article(url):
    """Crawl chi tiết 1 bài báo"""
    try:
        res = requests.get(url, headers=HEADERS, timeout=10)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")

        title = soup.select_one("h1.title-detail")
        title = title.get_text(strip=True) if title else "Unknown"

        date = soup.select_one("span.date")
        date = date.get_text(strip=True) if date else "Unknown"

        author = extract_author(soup)

        content_div = soup.select_one("article.fck_detail")
        paragraphs = content_div.find_all("p") if content_div else []
        content = " ".join(p.get_text(" ", strip=True) for p in paragraphs)

        tags = [a.get_text(strip=True) for a in soup.select("ul.breadcrumb li a")]

        return {
            "title": title,
            "url": url,
            "date": date,
            "author": author,
            "content": content,
            "tags": tags
        }
    except Exception as e:
        print(f"Lỗi crawl {url}: {e}")
        return None


def save_results():
    """Lưu dữ liệu ra file JSON"""
    global RESULTS, FILE_INDEX
    if RESULTS:
        filename = f"vnexpressT_bongda_part{FILE_INDEX}.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(RESULTS, f, ensure_ascii=False, indent=2)
        print(f"Đã lưu {len(RESULTS)} bài vào {filename}")
        RESULTS.clear()
        FILE_INDEX += 1


def crawl_search():
    """Crawl nhiều query cho đến khi đủ bài"""
    global CRAWLED_COUNT
    for query in QUERIES:
        page = 1
        while CRAWLED_COUNT < MAX_ARTICLES:
            params = {"q": query, "page": page}
            res = requests.get(SEARCH_URL, params=params, headers=HEADERS, timeout=10)

            try:
                data = res.json()
                articles = data.get("data", [])
            except Exception:
                print(f"[!] Trang {page} query '{query}' không trả JSON, thử HTML...")
                soup = BeautifulSoup(res.text, "html.parser")
                articles = [{"share_url": a["href"]} for a in soup.select("h3.title-news a[href]")]

            if not articles:
                break

            for item in articles:
                url = item.get("share_url")
                if not url or url in CRAWLED_URLS:
                    continue

                article = crawl_article(url)
                if article:
                    RESULTS.append(article)
                    CRAWLED_URLS.add(url)
                    CRAWLED_COUNT += 1
                    print(f"[{CRAWLED_COUNT}] {article['title']}")

                time.sleep(1)  # tránh bị chặn

                if len(RESULTS) >= SAVE_PER_FILE:
                    save_results()

                if CRAWLED_COUNT >= MAX_ARTICLES:
                    break

            page += 1

    # Lưu nốt phần còn lại
    save_results()
    print(f"Hoàn tất! Tổng số bài crawl: {CRAWLED_COUNT}")


def main():
    crawl_search()


if __name__ == "__main__":
    main()


[!] Trang 1 query 'bóng đá Việt Nam' không trả JSON, thử HTML...
[1] 'Ảo tưởng bóng đá Việt Nam vươn tầm khi giành vé dự VCK U23 châu Á'
[2] Liệu Malaysia có bị xử thua trận Việt Nam?
[3] Báo Nhật Bản: 'Thêm một lần muối mặt trước bóng đá Việt Nam'
[4] Đội tuyển Việt Nam thua đậm Nam Định
[5] Tuyển Việt Nam tìm lại niềm vui chiến thắng
[6] Unknown
[7] HLV Quảng Nam: 'Vấn nạn trọng tài kéo bóng đá Việt Nam đi xuống'
[8] HLV Kim Sang-sik đến với bóng đá Việt Nam thế nào
[9] Thủ tướng: Bóng đá Việt Nam cần nỗ lực vô địch châu Á, dự World Cup
[10] HLV Park Hang-seo: 'Đã tới lúc bóng đá Việt Nam vươn ra châu Á'
Đã lưu 10 bài vào vnexpressT_bongda_part1.json
[11] Agribank treo thưởng một tỷ đồng khi đội tuyển bóng đá Việt Nam vô địch
[12] Agribank thưởng đội tuyển bóng đá Việt Nam 1 tỷ đồng
[13] HLV Kim Sang-sik: 'Bóng đá Việt Nam đã trở lại'
[14] Unknown
[15] Unknown
[16] Cầu thủ Trẻ TP HCM xin lỗi vì làm xấu hình ảnh bóng đá Việt Nam
[17] Lối đá 'chuqua-chula-chuda' xói mòn tuyển bóng đá V